<a href="https://colab.research.google.com/github/unfortunate-code/Neural-Network-For-Taking-Derivatives/blob/main/Derivatives_FCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%cd drive/MyDrive

/content/drive/MyDrive


In [ ]:
import itertools
import random
import re
import torch
import torch.optim as optim
from torch import nn
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

In [ ]:
def simplify_equation(line):
  x, y = line.strip().split('=')
  matches = re.match('d\((.*)\)/d(.*)', x)
  return matches.group(1), matches.group(2), y

In [ ]:
with open('derivatives-takehome/train.txt', 'r') as f:
  data = [simplify_equation(line) for line in f.readlines()]

In [ ]:
def tokenize_equations(eqn, var, var_map):
  if not var_map:
    var_map[var] = 'var0'
    var_index = 1
  else:
    var_index = int(sorted(list(var_map.values()))[-1][-1]) + 1
  curr = ''
  tokens = []
  for i in range(len(eqn)):
    if 'a' <= eqn[i] <='z' or 'A' <= eqn[i] <= 'Z':
      curr += eqn[i]
    else:
      if curr:
        if len(curr) == 1:
          if curr in var_map:
            tokens.append(var_map[curr])
          else:
            var_map[curr] = 'var' + str(var_index)
            var_index += 1
            tokens.append(var_map[curr])
        else:
          tokens.append(curr)
        curr = ''
      tokens.append(eqn[i])
  return tokens, var_map

In [ ]:
tokenized_data = []
for x, v, y in data:
  x_tokens, var_map = tokenize_equations(x, v, {})
  y_tokens, var_map = tokenize_equations(y, v, var_map)
  tokenized_data.append((x_tokens, y_tokens, {v: k for k, v in var_map.items()}))

In [ ]:
input_vocabulary = set(itertools.chain.from_iterable([x for x, y, _ in tokenized_data]))
output_vocabulary = set(itertools.chain.from_iterable([y for x, y, _ in tokenized_data]))

In [ ]:
variables = set(itertools.chain.from_iterable([z.values() for _, _, z in tokenized_data]))

In [ ]:
print(variables)
print(input_vocabulary)
print(output_vocabulary)
print(len(variables))
print(len(input_vocabulary))
print(len(output_vocabulary))

{'x', 'a', 'm', 'k', 'u', 'y', 'n', 'c', 's', 'z', 'e', 'i', 'w', 'p', 't', 'b', 'r', 'v', 'o'}
{'cos', 'var0', ')', '1', '6', 'sin', '^', '4', '(', '+', '-', 'exp', '2', '9', '*', '3', '8', '0', '7', '5', 'var1'}
{'cos', 'var0', ')', '1', '6', 'sin', '^', '4', '(', '+', '-', 'exp', '2', '9', '*', '3', '8', '0', '7', '5', 'var1'}
19
21
21


In [ ]:
pad_token = '<pad>'
input_vocabulary.add(pad_token)
output_vocabulary.add(pad_token)
input_token_to_index = {}
input_index_to_token = {}
index = 0
for token in input_vocabulary:
  input_token_to_index[token] = index
  input_index_to_token[index] = token
  index += 1
output_token_to_index = {}
output_index_to_token = {}
index = 0
for token in output_vocabulary:
  output_token_to_index[token] = index
  output_index_to_token[index] = token
  index += 1

In [ ]:
all_indices = set(range(1000000))
train_indices = set(random.sample(all_indices, 9 * len(all_indices) // 10))
all_indices -= train_indices
val_indices = set(random.sample(all_indices, len(all_indices) // 2))
all_indices -= val_indices
test_indices = all_indices

In [ ]:
print(len(train_indices))
print(len(test_indices))
print(len(val_indices))

900000
50000
50000


In [ ]:
train_data = [tokenized_data[i] for i in train_indices]
test_data = [tokenized_data[i] for i in test_indices]
val_data = [tokenized_data[i] for i in val_indices]

In [ ]:
class DerivativesDataset(Dataset):
  def __init__(self, data):
    self.data = data
  
  def __len__(self):
    return len(self.data)
  
  def __getitem__(self, index):
    x, y, map = self.data[index]
    x = torch.tensor([input_token_to_index[token] for token in x] + (30 - len(x)) * [input_token_to_index[pad_token]])
    y = torch.tensor([output_token_to_index[token] for token in y] + (30 - len(x)) * [output_token_to_index[pad_token]])
    return x, y, map

In [ ]:
train_dataset = DerivativesDataset(train_data)
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True, pin_memory=True, num_workers=2)

In [ ]:
class DerivativesModel(nn.Module):
  def __init__(self, input_size, sequence_length, output_size, hidden_size):
    super(DerivativesModel, self).__init__()
    self.embedding_layer = nn.Embedding(input_size, hidden_size)
    self.linear1 = nn.Linear(hidden_size * sequence_length, hidden_size * sequence_length * 2)
    self.linear2 = nn.Linear(hidden_size * sequence_length * 2, hidden_size * sequence_length * 4)
    self.linear3 = nn.Linear(hidden_size * sequence_length * 4, hidden_size * sequence_length * 2)
    self.linear3 = nn.Linear(hidden_size * sequence_length * 2, output_size * sequence_length)
    torch.nn.init.kaiming_normal_(self.linear1.weight, mode='fan_in')
    torch.nn.init.kaiming_normal_(self.linear2.weight, mode='fan_in')
    torch.nn.init.kaiming_normal_(self.linear3.weight, mode='fan_in')
  
  def forward(self, x):
    embedding = self.embedding_layer(x)
    embedding = embedding.view(x.shape[0], -1)
    output = self.linear1(embedding)
    output = torch.nn.functional.relu(output)
    output = self.linear2(output)
    output = torch.nn.functional.relu(output)
    output = self.linear3(output)
    output = torch.nn.functional.relu(output)
    output = output.view(-1, output_size)
    return output

In [ ]:
hidden_size = 22
model = DerivativesModel(len(input_vocabulary), 30, len(output_vocabulary), hidden_size)
model = model.train().cuda()

RuntimeError: ignored

In [ ]:
learning_rate = 0.001
epochs = 100
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.CrossEntropyLoss()

In [ ]:
for epoch in range(epochs):
  print('Epoch', epoch)
  total_loss = []
  for x, y, map in tqdm(train_dataloader):
    x = x.cuda()
    y = y.cuda()
    optimizer.zero_grad()
    output = model(x)
    y = y.view(-1)
    loss = criterion(output, y)
    loss.backward()
    optimizer.step()
  print(sum(total_loss) / len(total_loss))

Epoch 0


100%|██████████| 220/220 [00:32<00:00,  6.72it/s]


3.1036582074718932
Epoch 1


100%|██████████| 220/220 [00:32<00:00,  6.83it/s]


1.621758813974798
Epoch 2


...

0.47717958994670423
Epoch 98


100%|██████████| 220/220 [00:31<00:00,  7.05it/s]


0.4760527797518999
Epoch 99


100%|██████████| 220/220 [00:31<00:00,  7.06it/s]

0.4751981512408063


In [ ]:
torch.save(model, 'fcnn-model.model')

In [ ]:
def get_eqn(y):
  res = []
  for i in range(len(y)):
    if y[i] == output_token_to_index[end_token]: return res
    if isinstance(y[i], int):
      res.append(output_index_to_token[y[i]])
    else:
      res.append(output_index_to_token[y[i].item()])
  return res

In [ ]:
def is_match(y, pred):
  for i in range(len(y)):
    if y[i] == output_token_to_index[end_token]: return True
    if i >= len(pred) or y[i] != pred[i]: return False
  return True

In [ ]:
def evaluate(dataloader, cutoff_length, encoder, decoder):
  encoder = encoder.eval()
  decoder = decoder.eval()
  count = correct_count = 0
  with torch.no_grad():
    for x, y, map in tqdm(dataloader):
      count += len(x)
      x, y = x.cuda(), y.cuda()
      _, (enc_hn, enc_cn) = encoder(x)
      for i in range(len(x)):
        prediction = []
        dec_h_i, dec_c_i = enc_hn[:, i, :].contiguous().unsqueeze(1), enc_cn[:, i, :].contiguous().unsqueeze(1)
        decoder_input = torch.IntTensor(1, 1).fill_(output_token_to_index[start_token]).cuda()
        decoded_indices = []
        for j in range(cutoff_length):
          pred, dec_h_i, dec_c_i = decoder(decoder_input, dec_h_i, dec_c_i)
          _, topi = pred.topk(1)
          topi = topi.item()
          if topi == output_token_to_index[end_token]:
            break
          else:
            decoded_indices.append(topi)
          decoder_input = torch.IntTensor(1, 1).fill_(topi).cuda()
        if is_match(y[i], decoded_indices): correct_count += 1
        # print(y[i])
        # print(decoded_indices)
        # print(get_eqn(x[i]))
        # print(get_eqn(y[i]))
        # print(get_eqn(decoded_indices))
        # return
  return correct_count / count

In [ ]:
val_dataset = DerivativesDataset(val_data)
val_dataloader = DataLoader(val_dataset, batch_size=4096, shuffle=False, pin_memory=True, num_workers=2, collate_fn=PadSequence())
evaluate(val_dataloader, 40, encoder, decoder)

100%|██████████| 13/13 [05:05<00:00, 23.52s/it]


0.01102

In [ ]:
get_eqn(val_dataset[0][1])

['4', '1', '6', 'exp', '^', '(', '1', '0', 'var1', ')', '*', 'var0', '^', '3']